In [1]:
import pandas as pd
import numpy as np

In [2]:
# HS6 to HS5 conversion
df = pd.read_excel('/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/HS2022toHS2017.xlsx', sheet_name='Conversions', dtype={'From HS2022': str, 'To HS2017': str})
df_corr = pd.read_excel('/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/HS2022toHS2017.xlsx', sheet_name='Correlations', dtype={'HS2022': str, 'HS2017': str})
df.head(2)
#df['From HS2022'].apply(len).unique() #only 6-digit codes

,From HS2022,To HS2017
0,010121,010121
1,010129,010129


#### Correlation analysis

In [3]:
# HS2017 level
df_corr_gr = df_corr.groupby(['HS2017'])['Relationship'].agg(list).reset_index()
df_corr_gr['Relationship_type'] = df_corr_gr['Relationship'].apply(lambda x: str(set(x)))
print(df_corr_gr['Relationship_type'].unique())
df_corr_gr['Corr_1:1'] = df_corr_gr['Relationship_type'].apply(lambda x: x == "{'1:1'}")
df_corr_gr['Corr_n:1_only'] = df_corr_gr['Relationship_type'].apply(lambda x: x == "{'n:1'}")


["{'1:1'}" "{'n:n', 'n:1'}" "{'1:n'}" "{'n:1'}" "{'n:n'}"]


In [4]:
df_corr_gr['Relationship_type'].value_counts() 

{'1:1'}           4097
{'n:n', 'n:1'}    1142
{'n:1'}             65
{'1:n'}             54
{'n:n'}             29
Name: Relationship_type, dtype: int64

#### Merging HS codes with Import sanctions

In [3]:
sanct_exp_refin = pd.read_csv('/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_1/sanct_imp_refin_v1.csv', index_col=0, dtype={'CN6':str})


In [20]:
# only 19 6digit codes are not covered by the 4digit codes (1 code is 2digit that's why 19 but not 20).
# So the analysis of "similar" goods is not promising here.
print(sanct_exp_refin['CN6_len'].value_counts())
giga = sanct_exp_refin.loc[sanct_exp_refin['CN6_len'] == 4, 'CN6'].unique()
print(sanct_exp_refin['CN6'].apply(lambda x: x[:4] in giga).sum() - (252 + 176 + 1))

4    252
6    176
2      1
Name: CN6_len, dtype: int64
-20


In [7]:
df[sanct_exp_refin.columns] = np.nan

sanct_codes = sanct_exp_refin['CN6'].values
for i in df.index:
    code = df.loc[i, 'From HS2022']
    if code in sanct_codes: df.loc[i, sanct_exp_refin.columns] = sanct_exp_refin.loc[sanct_exp_refin['CN6'] == code].squeeze(axis=0)
    elif code[:4] in sanct_codes: df.loc[i, sanct_exp_refin.columns] = sanct_exp_refin.loc[sanct_exp_refin['CN6'] == code[:4]].squeeze(axis=0)
    elif code[:2] in sanct_codes: df.loc[i, sanct_exp_refin.columns] = sanct_exp_refin.loc[sanct_exp_refin['CN6'] == code[:2]].squeeze(axis=0) 
    else: df.loc[i, sanct_exp_refin.columns] == np.nan 
        
df['Sanct'] = df['Date'].apply(lambda x: int(x==x))  
df['Issue1'] = (df['CN6_len'] < 6) & (df['CN6_Full_code_1-1'] != True)
HS6toHS5 = df.copy()

In [10]:
# select those generated from non fully sanctioned 2 or 4 digit codes, this issue is only for 87 codes on HS6 level
df.loc[(df['CN6_len'] < 6) & (df['CN6_Full_code_1-1'] != True)].shape
# select non-fully sanctioned 2 or 4 digit codes, this issue is only for 13 codes in cleaned sanctions list
sanct_exp_refin.loc[(sanct_exp_refin['CN6_len'] < 6) & (sanct_exp_refin['CN6_Full_code_1-1'] != True)].shape

(87, 13)

In [14]:
# check that CN6_Full_code_0-1_date_firstpartial always equals to Date for transitions codes
(sanct_exp_refin.loc[sanct_exp_refin['CN6_Full_code_0-1_proper'], 'CN6_Full_code_0-1_date_firstpartial'] == sanct_exp_refin.loc[sanct_exp_refin['CN6_Full_code_0-1_proper'], 'Date']).sum()


24

In [34]:
df_agg = df.loc[df['Date'].notna()].groupby(['Date'])['From HS2022'].count().reset_index()
df_agg['Sanct_prod'] = np.nan

# df_agg['Sanct_prod'].loc[0] = df_agg['From HS2022'].loc[0]
# for i in df_agg.index[1:]:
#     df_agg['Sanct_prod'].loc[i] = df_agg['Sanct_prod'].loc[i-1] + df_agg['From HS2022'].loc[i]
    
# fully sanctioned codes type of sanctioned codes    
df_agg_full = df.loc[(df['Date'].notna()) & (df['CN6_Full_code_1-1'] == True)].groupby(['Date'])['From HS2022'].count().reset_index()
df_agg_full['Sanct_prod'] = np.nan

df_agg_full['Sanct_prod'].loc[0] = df_agg_full['From HS2022'].loc[0]
for i in df_agg_full.index[1:]:
    df_agg_full['Sanct_prod'].loc[i] = df_agg_full['Sanct_prod'].loc[i-1] + df_agg_full['From HS2022'].loc[i] 
    
# partially sanctioned codes type of sanctioned codes
df_agg_part = df.loc[(df['Date'].notna()) & (df['CN6_Full_code_0-0'] == True)].groupby(['Date'])['From HS2022'].count().reset_index()
df_agg_part['Sanct_prod'] = np.nan

df_agg_part['Sanct_prod'].loc[0] = df_agg_part['From HS2022'].loc[0]
for i in df_agg_part.index[1:]:
    df_agg_part['Sanct_prod'].loc[i] = df_agg_part['Sanct_prod'].loc[i-1] + df_agg_part['From HS2022'].loc[i]
    
# transit sanctioned codes type of sanctioned codes    
df_agg_trans = df.loc[(df['Date'].notna()) & (df['CN6_Full_code_0-1_proper'] == True)].groupby(['Date'])['From HS2022'].count().reset_index()
df_agg_trans['Sanct_prod'] = np.nan

df_agg_trans['Sanct_prod'].loc[0] = df_agg_trans['From HS2022'].loc[0]
for i in df_agg_trans.index[1:]:
    df_agg_trans['Sanct_prod'].loc[i] = df_agg_trans['Sanct_prod'].loc[i-1] + df_agg_trans['From HS2022'].loc[i] 
    
    
# merge to one table    
df_agg_m = pd.merge(df_agg_full, df_agg_part, on='Date', how='outer')  
df_agg_m = pd.merge(df_agg_m, df_agg_trans, on='Date', how='outer')

df_agg_m.sort_values(by=['Date'], inplace=True)
df_agg_m.fillna(method='ffill', inplace=True)
df_agg_m.rename(columns={'Sanct_prod_x':'Full sanct.', 'Sanct_prod_y':'Partial sanct.', 'Sanct_prod':'Transitory sanct.'}, inplace=True)
df_agg_m.set_index('Date', inplace=True)    



/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [39]:
import plotly.express as px

plot1 = df_agg_m[['Full sanct.', 'Partial sanct.', 'Transitory sanct.']].stack().reset_index()
plot1.rename(columns={'level_1':'Type', 0:'Number of codes'}, inplace=True)

fig = px.area(plot1, x="Date", y="Number of codes", color="Type", category_orders={'Type':['Full sanct.', 'Partial sanct.', 'Transitory sanct.']}
             )
fig.show()

In [21]:
HS5_sanct = HS6toHS5.groupby(by=['To HS2017']).agg(list).reset_index()
HS5_sanct.head(2)

,To HS2017,From HS2022,CN6,Date,CN6_Full_code_0-0,CN6_Full_code_1-1,CN6_Full_code_0-1_proper,CN6_Full_code_0-1_deltaT_proper,CN6_Full_code_0-1_date_firstpartial,CN6_Full_code_0-1_date_firstfull,CN6_len,Sanct,Issue1
0,010121,[010121],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[0],[False]
1,010129,[010129],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[0],[False]


In [23]:
HS5_sanct['Date_HS5'] = np.nan
HS5_sanct['CN6_Full_code_0-0_HS5'] = np.nan
HS5_sanct['CN6_Full_code_1-1_HS5'] = np.nan
HS5_sanct['CN6_Full_code_0-1_proper_HS5'] = np.nan

HS5_sanct['CN6_Full_code_0-1_date_firstpartial_HS5'] = np.nan
HS5_sanct['CN6_Full_code_0-1_date_firstfull_HS5'] = np.nan
HS5_sanct['Sanct_HS5'] = np.nan


In [24]:
for i in HS5_sanct.index:
    if (len(HS5_sanct['From HS2022'].loc[i]) == 1) | (max(HS5_sanct['Sanct'].loc[i]) == 0): # if HS5 and HS6 match 1:1 or if 1:n and all HS6 are non-sanct
        HS5_sanct['Date_HS5'].loc[i] = HS5_sanct['Date'].loc[i][0]
        HS5_sanct['CN6_Full_code_0-0_HS5'].loc[i] = HS5_sanct['CN6_Full_code_0-0'].loc[i][0]        
        HS5_sanct['CN6_Full_code_1-1_HS5'].loc[i] = HS5_sanct['CN6_Full_code_1-1'].loc[i][0]
        HS5_sanct['CN6_Full_code_0-1_proper_HS5'].loc[i] = HS5_sanct['CN6_Full_code_0-1_proper'].loc[i][0]
        
        HS5_sanct['CN6_Full_code_0-1_date_firstpartial_HS5'].loc[i] = HS5_sanct['CN6_Full_code_0-1_date_firstpartial'].loc[i][0]
        HS5_sanct['CN6_Full_code_0-1_date_firstfull_HS5'].loc[i] = HS5_sanct['CN6_Full_code_0-1_date_firstfull'].loc[i][0]
        HS5_sanct['Sanct_HS5'].loc[i] = HS5_sanct['Sanct'].loc[i][0]
        
    elif min(HS5_sanct['Sanct'].loc[i]) != max(HS5_sanct['Sanct'].loc[i]): # if 1:n and HS5 groups both sanctioned and non-sanct HS6 codes
        HS5_sanct['Date_HS5'].loc[i] = min([x for x in HS5_sanct['Date'].loc[i] if x==x]) # clean from nans and take the earliest date
        
        HS5_sanct['CN6_Full_code_0-0_HS5'].loc[i] = True    # 0 since there are non-sanctioned HS6 codes in HS5 group
        HS5_sanct['CN6_Full_code_1-1_HS5'].loc[i] = False
        HS5_sanct['CN6_Full_code_0-1_proper_HS5'].loc[i] = False
        
        HS5_sanct['CN6_Full_code_0-1_date_firstpartial_HS5'].loc[i] = np.nan
        HS5_sanct['CN6_Full_code_0-1_date_firstfull_HS5'].loc[i] = np.nan
        HS5_sanct['Sanct_HS5'].loc[i] = 1
        
    elif min(HS5_sanct['Sanct'].loc[i]) == 1: # if 1:n and all HS6 are sanct
        # consider 4 cases (which should be sufficient to cover all possibilities):
        list0_0 = HS5_sanct['CN6_Full_code_0-0'].loc[i]
        list1_1 = HS5_sanct['CN6_Full_code_1-1'].loc[i]
        list0_1 = HS5_sanct['CN6_Full_code_0-1_proper'].loc[i]
        dates = HS5_sanct['Date'].loc[i]
        date_firstpartial = HS5_sanct['CN6_Full_code_0-1_date_firstpartial'].loc[i]
        date_firstfull = HS5_sanct['CN6_Full_code_0-1_date_firstfull'].loc[i]
        
        # 1st - if at least one True in 0-0, then the whole HS5 should be partial sanct.
        if True in list0_0:
            HS5_sanct['CN6_Full_code_0-0_HS5'].loc[i] = True
            HS5_sanct['CN6_Full_code_1-1_HS5'].loc[i] = False
            HS5_sanct['CN6_Full_code_0-1_proper_HS5'].loc[i] = False            
            
            HS5_sanct['Date_HS5'].loc[i] = min(HS5_sanct['Date'].loc[i]) # take the earliest date, as all are sanctions at various degree
            HS5_sanct['Sanct_HS5'].loc[i] = 1            
            
        # 2nd - if all are True in 1-1, then the whole HS5 is either full sanct. or transit (if other parts of the product become fully sanct. later)
        elif all(list1_1) == True:
            if min(dates) != max(dates):            
                HS5_sanct['CN6_Full_code_0-0_HS5'].loc[i] = False
                HS5_sanct['CN6_Full_code_1-1_HS5'].loc[i] = False
                HS5_sanct['CN6_Full_code_0-1_proper_HS5'].loc[i] = True
                
                HS5_sanct['Date_HS5'].loc[i] = min(dates)
                HS5_sanct['CN6_Full_code_0-1_date_firstpartial_HS5'].loc[i] = min(dates)
                HS5_sanct['CN6_Full_code_0-1_date_firstfull_HS5'].loc[i] = max(dates)
            else:
                HS5_sanct['CN6_Full_code_0-0_HS5'].loc[i] = False
                HS5_sanct['CN6_Full_code_1-1_HS5'].loc[i] = True
                HS5_sanct['CN6_Full_code_0-1_proper_HS5'].loc[i] = False
                
                HS5_sanct['Date_HS5'].loc[i] = HS5_sanct['Date'].loc[i][0]            
        
            HS5_sanct['Sanct_HS5'].loc[i] = 1
        
        # 3rd - if all are True in 0-1, then the whole HS5 is transit
        elif all(list0_1) == True:            
            HS5_sanct['CN6_Full_code_0-0_HS5'].loc[i] = False
            HS5_sanct['CN6_Full_code_1-1_HS5'].loc[i] = False
            HS5_sanct['CN6_Full_code_0-1_proper_HS5'].loc[i] = True
            
            HS5_sanct['Date_HS5'].loc[i] = min(date_firstpartial)
            HS5_sanct['CN6_Full_code_0-1_date_firstpartial_HS5'].loc[i] = min(date_firstpartial)
            HS5_sanct['CN6_Full_code_0-1_date_firstfull_HS5'].loc[i] = max(date_firstfull)
            HS5_sanct['Sanct_HS5'].loc[i] = 1

        # 4th - if there are both fully sanct. and transit, then the whole HS5 is transit
        elif (True in list1_1) & (True in list0_1):            
            HS5_sanct['CN6_Full_code_0-0_HS5'].loc[i] = False
            HS5_sanct['CN6_Full_code_1-1_HS5'].loc[i] = False
            HS5_sanct['CN6_Full_code_0-1_proper_HS5'].loc[i] = True        
            
            HS5_sanct['Date_HS5'].loc[i] = min(dates)
            HS5_sanct['CN6_Full_code_0-1_date_firstpartial_HS5'].loc[i] = min(dates)
            HS5_sanct['CN6_Full_code_0-1_date_firstfull_HS5'].loc[i] = max(dates + [x for x in date_firstfull if x==x])
            HS5_sanct['Sanct_HS5'].loc[i] = 1

        
#         HS5_sanct['Date_HS5'].loc[i] = min(HS5_sanct['Date'].loc[i]) # take the earliest date 
#         HS5_sanct['Full_code_HS5'].loc[i] = min(HS5_sanct['CN6_Full_code'].loc[i]) # 1 if all HS6 codes are fully sanct, 0 if there is at least one non-fully
#         HS5_sanct['Sanct_HS5'].loc[i] = 1    

/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
# make Issue1 column (4 or 2 digit codes which are partially sanct.) 87 of such codes
HS5_sanct['Issue1_HS5'] = HS5_sanct['Issue1'].apply(lambda x: True in x)

In [ ]:
# 122 HS2017 codes are associated with multiple HS2022 codes
HS5_sanct.loc[HS5_sanct['From HS2022'].apply(len) > 1].shape

In [40]:
df_agg = HS5_sanct.loc[HS5_sanct['Date_HS5'].notna()].groupby(['Date_HS5'])['To HS2017'].count().reset_index()
df_agg['Sanct_prod_HS5'] = np.nan
    
# fully sanctioned codes type of sanctioned codes    
df_agg_full = HS5_sanct.loc[(HS5_sanct['Date_HS5'].notna()) & (HS5_sanct['CN6_Full_code_1-1_HS5'] == True)].groupby(['Date_HS5'])['To HS2017'].count().reset_index()
df_agg_full['Sanct_prod_HS5'] = np.nan

df_agg_full['Sanct_prod_HS5'].loc[0] = df_agg_full['To HS2017'].loc[0]
for i in df_agg_full.index[1:]:
    df_agg_full['Sanct_prod_HS5'].loc[i] = df_agg_full['Sanct_prod_HS5'].loc[i-1] + df_agg_full['To HS2017'].loc[i] 
    
# partially sanctioned codes type of sanctioned codes
df_agg_part = HS5_sanct.loc[(HS5_sanct['Date_HS5'].notna()) & (HS5_sanct['CN6_Full_code_0-0_HS5'] == True)].groupby(['Date_HS5'])['To HS2017'].count().reset_index()
df_agg_part['Sanct_prod_HS5'] = np.nan

df_agg_part['Sanct_prod_HS5'].loc[0] = df_agg_part['To HS2017'].loc[0]
for i in df_agg_part.index[1:]:
    df_agg_part['Sanct_prod_HS5'].loc[i] = df_agg_part['Sanct_prod_HS5'].loc[i-1] + df_agg_part['To HS2017'].loc[i]
    
# transit sanctioned codes type of sanctioned codes    
df_agg_trans = HS5_sanct.loc[(HS5_sanct['Date_HS5'].notna()) & (HS5_sanct['CN6_Full_code_0-1_proper_HS5'] == True)].groupby(['Date_HS5'])['To HS2017'].count().reset_index()
df_agg_trans['Sanct_prod_HS5'] = np.nan

df_agg_trans['Sanct_prod_HS5'].loc[0] = df_agg_trans['To HS2017'].loc[0]
for i in df_agg_trans.index[1:]:
    df_agg_trans['Sanct_prod_HS5'].loc[i] = df_agg_trans['Sanct_prod_HS5'].loc[i-1] + df_agg_trans['To HS2017'].loc[i] 
    
    
# merge to one table    
df_agg_m = pd.merge(df_agg_full, df_agg_part, on='Date_HS5', how='outer')  
df_agg_m = pd.merge(df_agg_m, df_agg_trans, on='Date_HS5', how='outer')

df_agg_m.sort_values(by=['Date_HS5'], inplace=True)
df_agg_m.fillna(method='ffill', inplace=True)
df_agg_m.rename(columns={'Sanct_prod_HS5_x':'Full sanct.', 'Sanct_prod_HS5_y':'Partial sanct.', 'Sanct_prod_HS5':'Transitory sanct.'}, inplace=True)
df_agg_m.set_index('Date_HS5', inplace=True)    

/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
import plotly.express as px

plot1 = df_agg_m[['Full sanct.', 'Partial sanct.', 'Transitory sanct.']].stack().reset_index()
plot1.rename(columns={'level_1':'Type', 0:'Number of codes'}, inplace=True)

fig = px.area(plot1, x="Date_HS5", y="Number of codes", color="Type", category_orders={'Type':['Full sanct.', 'Partial sanct.', 'Transitory sanct.']}
             )
fig.show()

In [42]:
HS5_sanct_short = HS5_sanct[['To HS2017', 'From HS2022',
       'Date_HS5', 'CN6_Full_code_0-0_HS5', 'CN6_Full_code_1-1_HS5',
       'CN6_Full_code_0-1_proper_HS5',
       'CN6_Full_code_0-1_date_firstpartial_HS5',
       'CN6_Full_code_0-1_date_firstfull_HS5', 'Sanct_HS5', 'Issue1_HS5']]

In [43]:
HS5_sanct_short['CN6_Full_code_0-0_HS5'] = HS5_sanct_short['CN6_Full_code_0-0_HS5'].apply(lambda x: int(x) if x==x else x)
HS5_sanct_short['CN6_Full_code_1-1_HS5'] = HS5_sanct_short['CN6_Full_code_1-1_HS5'].apply(lambda x: int(x) if x==x else x)
HS5_sanct_short['CN6_Full_code_0-1_proper_HS5'] = HS5_sanct_short['CN6_Full_code_0-1_proper_HS5'].apply(lambda x: int(x) if x==x else x)


/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [45]:
HS5_sanct_short['Sanct_type'] = np.nan
for i in HS5_sanct_short.index:
    code0_0 = HS5_sanct_short.loc[i, 'CN6_Full_code_0-0_HS5']
    code1_1 = HS5_sanct_short.loc[i, 'CN6_Full_code_1-1_HS5']    
    code0_1 = HS5_sanct_short.loc[i, 'CN6_Full_code_0-1_proper_HS5']
    
    if code0_0 != code0_0: HS5_sanct_short.loc[i, 'Sanct_type'] = 'Non_sanctioned'
    elif code0_0 == 1: HS5_sanct_short.loc[i, 'Sanct_type'] = 'Partial'
    elif code1_1 == 1: HS5_sanct_short.loc[i, 'Sanct_type'] = 'Full'
    else: HS5_sanct_short.loc[i, 'Sanct_type'] = 'Transitory'

/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [65]:
HS5_sanct_short_ext = pd.merge(HS5_sanct_short, df_corr_gr[['HS2017', 'Corr_1:1', 'Corr_n:1_only']], left_on='To HS2017', right_on='HS2017', how='left')
HS5_sanct_short_ext['Issue2_HS5'] = (df_corr_gr['Corr_1:1'] | df_corr_gr['Corr_n:1_only']).apply(lambda x: not x)


In [66]:
HS5_sanct_short_ext['Issue1_HS5'] = HS5_sanct_short_ext['Issue1_HS5'].apply(lambda x: int(x) if x==x else x)
HS5_sanct_short_ext['Corr_1:1'] = HS5_sanct_short_ext['Corr_1:1'].apply(lambda x: int(x) if x==x else x)
HS5_sanct_short_ext['Corr_n:1_only'] = HS5_sanct_short_ext['Corr_n:1_only'].apply(lambda x: int(x) if x==x else x)
HS5_sanct_short_ext['Issue2_HS5'] = HS5_sanct_short_ext['Issue2_HS5'].apply(lambda x: int(x) if x==x else x)


In [67]:
HS5_sanct_short_ext = HS5_sanct_short_ext[['To HS2017', 'From HS2022', 'Date_HS5', 
                                           'CN6_Full_code_0-0_HS5','CN6_Full_code_1-1_HS5', 'CN6_Full_code_0-1_proper_HS5',
                                           'CN6_Full_code_0-1_date_firstpartial_HS5','CN6_Full_code_0-1_date_firstfull_HS5', 
                                           'Sanct_HS5', 'Sanct_type',
                                           'Issue1_HS5', 'Issue2_HS5',
                                           'Corr_1:1','Corr_n:1_only']]
HS5_sanct_short_ext.head(2)

,To HS2017,From HS2022,Date_HS5,CN6_Full_code_0-0_HS5,CN6_Full_code_1-1_HS5,CN6_Full_code_0-1_proper_HS5,CN6_Full_code_0-1_date_firstpartial_HS5,CN6_Full_code_0-1_date_firstfull_HS5,Sanct_HS5,Sanct_type,Issue1_HS5,Issue2_HS5,Corr_1:1,Corr_n:1_only
0,010121,[010121],NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,1,0
1,010129,[010129],NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,1,0


In [68]:
HS5_sanct_short_ext.rename(columns={'To HS2017':'HS2017', 'From HS2022':'HS2022', 
                                    'CN6_Full_code_0-0_HS5':'Partial_HS5', 'CN6_Full_code_1-1_HS5':'Full_HS5', 'CN6_Full_code_0-1_proper_HS5':'Transitory_HS5',
                                   'CN6_Full_code_0-1_date_firstpartial_HS5':'Transitory_firstpatial_HS5', 'CN6_Full_code_0-1_date_firstfull_HS5':'Transitory_firstfull_HS5',
                                   }, inplace=True)


In [70]:
HS5_sanct_short_ext.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_2/HS5_sanct_short_ext_imp_v1.csv")


In [69]:
# HS5_sanct_short_ext.columns
# HS5_sanct_short

HS5_sanct_short_ext

,HS2017,HS2022,Date_HS5,Partial_HS5,Full_HS5,Transitory_HS5,Transitory_firstpatial_HS5,Transitory_firstfull_HS5,Sanct_HS5,Sanct_type,Issue1_HS5,Issue2_HS5,Corr_1:1,Corr_n:1_only
0,010121,[010121],NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,1,0
1,010129,[010129],NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,1,0
2,010130,[010130],NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,1,0
3,010190,[010190],NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,1,0
4,010221,[010221],NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5357,970300,"[970310, 970390]",NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,0,1
5358,970400,[970400],NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,1,0
5359,970500,"[970510, 970521, 970522, 970529, 970531, 970539]",NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,0,1
5360,970600,"[970610, 970690]",NaN,NaN,NaN,NaN,NaN,NaN,0.0,Non_sanctioned,0,0,0,1


#### Add similare goods indicator

In [2]:
HS5_sanct_short_ext = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_2/HS5_sanct_short_ext_imp_v1.csv", index_col=0, dtype={'HS2017': str})


In [4]:
HS5_sanct_short_ext['4digit'] = HS5_sanct_short_ext['HS2017'].apply(lambda x: x[:4])
df_similar = HS5_sanct_short_ext.loc[HS5_sanct_short_ext['Sanct_HS5'] == 1, ['4digit', 'Date_HS5']].groupby('4digit')['Date_HS5'].agg(list).apply(min).reset_index()
df_similar.rename(columns={'Date_HS5':'Date_HS5_similar'}, inplace=True)
HS5_sanct_short_ext = pd.merge(HS5_sanct_short_ext, df_similar, on='4digit', how='left')
HS5_sanct_short_ext.loc[HS5_sanct_short_ext['Sanct_HS5'] == 1, 'Date_HS5_similar'] = np.nan

In [5]:
# there are 37 of similar goods
print(HS5_sanct_short_ext['Date_HS5_similar'].notna().sum())

37


In [6]:
HS5_sanct_short_ext.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_2/HS5_sanct_short_ext_imp_v2.csv")


In [53]:
EU_imp = pd.read_csv('/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Input_Script_2/TradeData_EUimpAnn.csv', dtype={'CmdCode': str})

EU_imp = EU_imp[['PartnerISO', 'CmdCode', 'PrimaryValue']]
EU_imp = pd.merge(EU_imp, EU_imp.groupby('PartnerISO')['PrimaryValue'].sum().reset_index(), on='PartnerISO', how='left')
EU_imp['Share'] = EU_imp['PrimaryValue_x'] / EU_imp['PrimaryValue_y']

In [54]:
EU_imp_W = EU_imp.loc[EU_imp['PartnerISO'] == 'W00']
EU_imp_R = EU_imp.loc[EU_imp['PartnerISO'] == 'RUS']
EU_imp_W.rename(columns={'Share':'Share_World'}, inplace=True)
EU_imp_R.rename(columns={'Share':'Share_Rus'}, inplace=True)

/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [55]:
HS5_sanct_short_ext = pd.merge(HS5_sanct_short_ext, EU_imp_R[['CmdCode', 'PrimaryValue_y', 'Share_Rus']], left_on='HS2017', right_on='CmdCode', how='left')
HS5_sanct_short_ext = pd.merge(HS5_sanct_short_ext, EU_imp_W[['CmdCode', 'PrimaryValue_y', 'Share_World']], left_on='HS2017', right_on='CmdCode', how='left')


In [56]:
HS5_sanct_short_ext = pd.merge(HS5_sanct_short_ext, df_corr_gr[['HS2017', 'Relationship_type']], on='HS2017', how='left')


#### Make a crosstab table

In [57]:
HS5_sanct_short_ext['Relationship_type'].value_counts()

{'1:1'}           4097
{'n:n', 'n:1'}    1142
{'n:1'}             65
{'1:n'}             39
{'n:n'}             19
Name: Relationship_type, dtype: int64

In [58]:
stat1 = HS5_sanct_short_ext.groupby('Relationship_type')[['Share_World', 'Share_Rus', 'Sanct_HS5']].sum()
stat1['Sanct_HS5_share'] = stat1['Sanct_HS5'] / stat1['Sanct_HS5'].sum()

stat1 = pd.concat([stat1, HS5_sanct_short_ext['Relationship_type'].value_counts()], axis=1, join="outer")
stat1['Relationship_type_share'] = stat1['Relationship_type'] / stat1['Relationship_type'].sum()

In [59]:
stat1 = stat1[['Relationship_type', 'Relationship_type_share', 'Sanct_HS5', 'Sanct_HS5_share', 'Share_World', 'Share_Rus']]
stat1.reindex(index=["{'1:1'}", "{'n:1'}", "{'n:n', 'n:1'}", "{'1:n'}", "{'n:n'}"])

,Relationship_type,Relationship_type_share,Sanct_HS5,Sanct_HS5_share,Share_World,Share_Rus
{'1:1'},4097,0.764081,1187.0,0.701537,0.595766,0.970858
{'n:1'},65,0.012122,30.0,0.017730,0.014020,0.002105
"{'n:n', 'n:1'}",1142,0.212980,458.0,0.270686,0.375744,0.026475
{'1:n'},39,0.007273,9.0,0.005319,0.004135,0.000218
{'n:n'},19,0.003543,8.0,0.004728,0.009154,0.000251


In [61]:
print(stat1.to_latex(index=True,
                  float_format="{:.2f}".format,
)) 

\begin{tabular}{lrrrrrr}
\toprule
{} &  Relationship\_type &  Relationship\_type\_share &  Sanct\_HS5 &  Sanct\_HS5\_share &  Share\_World &  Share\_Rus \\
\midrule
\{'1:1'\}        &               4097 &                     0.76 &    1187.00 &             0.70 &         0.60 &       0.97 \\
\{'1:n'\}        &                 39 &                     0.01 &       9.00 &             0.01 &         0.00 &       0.00 \\
\{'n:1'\}        &                 65 &                     0.01 &      30.00 &             0.02 &         0.01 &       0.00 \\
\{'n:n', 'n:1'\} &               1142 &                     0.21 &     458.00 &             0.27 &         0.38 &       0.03 \\
\{'n:n'\}        &                 19 &                     0.00 &       8.00 &             0.00 &         0.01 &       0.00 \\
\bottomrule
\end{tabular}

